In [2]:
from __future__ import division 

from pyomo.environ import *
from pyomo.opt import SolverFactory

model=AbstractModel()

opt=SolverFactory('ipopt')

#Sets
model.i=Set(initialize=['conv','effic','hybrid','elec'],ordered=True,doc='car types')
model.iter=RangeSet(0,1000)
model.ir=[]
model.j=Set(initialize=model.i) #alias(i,j) or alias(j,i)

#Parameters
model.pr=Param(model.i,initialize={'conv':12,'effic':15,'hybrid':25,'elec':45},mutable=True,doc='levelized cost of car EUR per 100km')
model.base=Param(model.i,initialize={'conv':0.80,'effic':0.07,'hybrid':0.10,'elec':0.03},mutable=True,doc='shares of car types in base year')
model.em=Param(model.i,initialize={'conv':10,'effic':5,'hybrid':1,'elec':0},mutable=True,doc='emissions kgCO2 per 100km')
model.type1=Param(model.i,initialize={'conv':0,'effic':0,'hybrid':0,'elec':1},mutable=True,doc='compliance with promoted car type')
model.gamma=Param(initialize=-2.5,mutable=True,doc='exponent of discrete choice')
model.lamdalo=Param(initialize=0,mutable=True,doc='min dual value of emission constraint measured as additional cost')
model.lamda=Param(mutable=True,doc='dual value of emission constraint measured as additional cost')
model.lamdaup=Param(initialize=1000,mutable=True,doc='max dual value of emission constraint measured as additional cost')
model.mulo=Param(initialize=0,mutable=True,doc='dual value of desired car type measured as additional cost')
model.mu=Param(mutable=True,doc='dual value of desired car type measured as additional cost')
model.muup=Param(initialize=1000,mutable=True,doc='dual value of desired car type measured as additional cost')
model.CO2goal=Param(initialize=0.9,mutable=True,doc='target of emissions')
model.cargoal=Param(initialize=0.50,mutable=True,doc='target of share of promoted car type')
model.CO2perf=Param(mutable=True,doc='performance for emissions')
model.carperf=Param(mutable=True,doc='performance in terms of share of promoted car type')
model.count=Param(mutable=True,doc='counter of model runs -iterations')
model.count1=Param(mutable=True)
model.count2=Param(initialize=0,mutable=True)
model.conv=Param(mutable=True,doc='convergence criterion')
model.tolerance=Param(initialize=10**-3,mutable=True,doc='tolerance of convergence')
model.maxiter=Param(initialize=200,mutable=True)
model.report=dict()

#Variables
model.shares=Var(model.i,bounds=(0.0,None),doc='shares of technologies')
model.cost=Var(model.i,bounds=(0.0,None),doc='generalized unit cost')

instance=model.create_instance()    #MODEL instance constraction. From now on model. will be instance.

instance.lamda=value(instance.lamdalo)  #dinw arxiki timi sto lamda

instance.mu=value(instance.mulo)        # to idio

#Asignments
for i in instance.i:
    instance.shares[i]=value(instance.base[i])
    
for i in instance.i:
    instance.cost[i]=value(instance.pr[i])+ value(instance.lamda)*value(instance.em[i])+value(instance.mu)*value(1-instance.type1[i])

# Objective and Constraints equations
def eqobj_rule(instance):
    return 0
                                                                                                           
instance.obj=Objective(rule=eqobj_rule,doc='dummy objective')

def eq1_rule(instance,i):
    return instance.shares[i]==instance.base[i]*instance.cost[i]**instance.gamma/sum(instance.base[j]*instance.cost[j]**instance.gamma for j in instance.j)

instance.eq1=Constraint(instance.i,rule=eq1_rule)

def eq2_rule(instance,i):
    return instance.cost[i]==instance.pr[i]+instance.lamda*instance.em[i]+instance.mu*(1-instance.type1[i])

instance.eq2=Constraint(instance.i,rule=eq2_rule)
  
instance.count=0
instance.conv=1000

def twocarpolicies_sub(model,instance):    #synartisi pou 
    
    from sys import float_info as f
    eps=f.epsilon
    for i in instance.iter: 
        if i==value(instance.count):
            instance.ir.append(i)       # to ir einai mia keni lista. leitourgei san ena keno set sto gams
        else:                           # alla den sxetizete me to modelo apla metraei poses fores tha lythei to modelo 
            pass                        # na ftasoume se apotelesma
        
    results= opt.solve(instance)        # edw lynete to modelo kai oi variables cost kai shares pernoun diaforetikes times 

    instance.solutions.load_from(results)
    
    instance.CO2perf=sum(instance.shares[i]*instance.em[i] for i in instance.i)     # me afta ta sum allazoun se kathe lysi tou
    instance.carperf=sum(instance.shares[i]*instance.type1[i]for i in instance.i)   # montelou oi varibles CO2perf kai carperf 
    
    instance.report[instance.ir[0],'count','']=instance.count+eps                          # to report einai ena keno dictionary
    instance.report[instance.ir[0],'CO2perf','']=instance.CO2perf+eps                      # antistoixei se keni parametro tou gams
    instance.report[instance.ir[0],'CO2diff','']=instance.CO2perf-instance.CO2goal+eps     # einai ki afto ektos modelou kanei 
    instance.report[instance.ir[0],'CO2sinth','']=instance.CO2perf/instance.CO2goal-1+eps  # report tis parametrous dipla
    instance.report[instance.ir[0],'carperf','']=instance.carperf+eps
    instance.report[instance.ir[0],'cardiff','']=instance.carperf-instance.cargoal+eps
    instance.report[instance.ir[0],'carsinth','']=instance.carperf/instance.cargoal-1+eps
    instance.report[instance.ir[0],'conv','']=instance.conv+eps
    instance.report[instance.ir[0],'lamda','']=instance.lamda+eps
    instance.report[instance.ir[0],'lamdalo','']=instance.lamdalo+eps
    instance.report[instance.ir[0],'lamdaup','']=instance.lamdaup+eps
    instance.report[instance.ir[0],'mu','']=instance.mu+eps
    instance.report[instance.ir[0],'mulo','']=instance.mulo+eps
    instance.report[instance.ir[0],'muup','']=instance.muup+eps
    for i in instance.i:
        instance.report[instance.ir[0],'cost',i]=instance.cost[i]+eps
        instance.report[instance.ir[0],'shares',i]=instance.shares[i]+eps
    instance.ir=[]                                                          # edw diagrafw to  periexomeno tou ir wste na 
                                                                            # exw se kathe lisi mono ton aritho tou count mesa
twocarpolicies_sub(model,instance) #1st solve

display(instance)

instance.count=0
# Algorithmos epislisis
while value(instance.count)<=value(instance.maxiter):
    while abs(value(instance.CO2perf/instance.CO2goal-1))>value(instance.tolerance) or abs(value(instance.carperf/instance.cargoal-1))>value(instance.tolerance):
        if abs(value(instance.CO2perf/instance.CO2goal-1))>value(instance.tolerance):   #3
            instance.conv=1000
            if value(instance.count)==0:
                instance.lamdalo=0
                instance.lamdaup=1000
            else:
                instance.lamdalo=value(instance.lamda/2.5)
                instance.lamdaup=value(instance.lamda*3.5)
            instance.lamda=value((instance.lamdaup+instance.lamdalo)/2)
            while abs(value(instance.CO2perf/instance.CO2goal-1))>value(instance.tolerance):   #5
                instance.count=value(instance.count+1)
                twocarpolicies_sub(model,instance)
                if value(instance.CO2perf-instance.CO2goal)<=0:
                    instance.lamdaup=value(instance.lamda)
                else:
                    instance.lamdalo=value(instance.lamda)
                instance.lamda=value((instance.lamdaup+instance.lamdalo)/2)
                instance.conv=abs(value(instance.CO2perf/instance.CO2goal-1))
        if value(instance.carperf-instance.cargoal)>=0:
            instance.conv=0 
        else:
            if abs(value(instance.carperf/instance.cargoal-1))>value(instance.tolerance):  #8
                instance.conv=1000
                if value(instance.count2)==0:
                    instance.mulo=0
                    instance.muup=1000
                else:
                    instance.mulo=value(instance.mu/2.5)
                    instance.muup=value(instance.mu*3.5)
                instance.mu=value((instance.muup+instance.mulo)/2)
                while abs(value(instance.carperf/instance.cargoal-1))>value(instance.tolerance):
                    instance.count=value(instance.count+1)
                    instance.count2=value(instance.count2+1)
                    twocarpolicies_sub(model,instance)
                    if value(instance.carperf-instance.cargoal)>0:
                        instance.muup=value(instance.mu)
                    else:
                        instance.mulo=value(instance.mu)
                    instance.mu=value((instance.muup+instance.mulo)/2)
                    instance.conv=abs(value(instance.carperf/instance.cargoal-1))
                    
    instance.count=value(instance.count+1)

for k in sorted(instance.report):
    print '{x}:{y}'.format(x=k,y=value(instance.report[k]))
display(instance)

2.22044604925e-16
Model unknown

  Variables:
    shares : shares of technologies
        Size=4, Index=i
        Key    : Lower : Value            : Upper : Fixed : Stale : Domain
          conv :   0.0 :    0.93334236047 :  None : False : False :  Reals
         effic :   0.0 :  0.0467491800001 :  None : False : False :  Reals
          elec :   0.0 : 0.00128526912655 :  None : False : False :  Reals
        hybrid :   0.0 :  0.0186231904035 :  None : False : False :  Reals
    cost : generalized unit cost
        Size=4, Index=i
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
          conv :   0.0 :  12.0 :  None : False : False :  Reals
         effic :   0.0 :  15.0 :  None : False : False :  Reals
          elec :   0.0 :  45.0 :  None : False : False :  Reals
        hybrid :   0.0 :  25.0 :  None : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.0

  Constraints:
    eq1 :